In [ ]:
#r "nuget: Thinksharp.TimeFlow"
#r "nuget: Thinksharp.TimeFlow.Interactive"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"

// Usings
using Thinksharp.TimeFlow;

Installed Packages Thinksharp.TimeFlow, 0.23.0 Thinksharp.TimeFlow.Interactive, 0.23.0 XPlot.Plotly, 4.0.3 XPlot.Plotly.Interactive, 4.0.3

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Loading extensions from `ThinkSharp.TimeFlow.Interactive.dll`

TimeFlow formatter registered.

---

## Time Series Creation

Time series can be created using the available methods of the static **TimeSeries.Factory** property.
The factory is of type **ITimeSeriesFactory**, which can be extended via extension methods. Therefore third party libraries may provide advanced factory methods or user can write it's own use-case specific ones.

### TimeSeries.Factory.FromValue

In [ ]:
// FromValue creates a time series with a defined value for each time point:
var ts = TimeSeries.Factory.FromValue(10,
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 01, 05), // end
    Period.Day);

ts

2021-01-01 00:00:00Z,10
2021-01-02 00:00:00Z,10
2021-01-03 00:00:00Z,10
2021-01-04 00:00:00Z,10
2021-01-05 00:00:00Z,10


### TimeSeries.Factory.FromValues

In [ ]:
// FromValues creates a time series from an enumerable of values:
var values = new decimal?[] {1, 2, 3, 4, 5, 6, 7, 8, 9};
var ts = TimeSeries.Factory.FromValues(values,
    new DateTime(2021, 01, 01), // start
    Period.Day);

ts

2021-01-01 00:00:00Z,1
2021-01-02 00:00:00Z,2
2021-01-03 00:00:00Z,3
2021-01-04 00:00:00Z,4
2021-01-05 00:00:00Z,5
2021-01-06 00:00:00Z,6
2021-01-07 00:00:00Z,7
2021-01-08 00:00:00Z,8
2021-01-09 00:00:00Z,9


### TimeSeries.Factory.FromGenerator

In [ ]:
// FromGenerator creates a time series with a value created as function of the time point:
var ts = TimeSeries.Factory.FromGenerator(
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 01, 05), // end
    Period.Day,
    tp => tp.Day); // tp: time point as DateTimeOffset

ts

2021-01-01 00:00:00Z,1
2021-01-02 00:00:00Z,2
2021-01-03 00:00:00Z,3
2021-01-04 00:00:00Z,4
2021-01-05 00:00:00Z,5


---
## Time Series Transformation

### Combining Time Series via Operators

In [ ]:
// time series can be combined using default operators:
var a = TimeSeries.Factory.FromValue(10,
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 01, 05), // end
    Period.Day);

var b = TimeSeries.Factory.FromGenerator(
    new DateTime(2021, 01, 01), // start   
    new DateTime(2021, 01, 05), // end 
    Period.Day,
    tp => tp.Day);

var tf = new TimeFrame();
tf["a"] = a;
tf["b"] = b;
tf["a + b"] = a + b;
tf["a - b"] = a - b;
tf["a * b"] = a * b;
tf["a / b"] = a / b;
tf["a * 12"] = a * 12;
tf


2021-01-01 00:00:00Z,10,1,11,9,10,10,120
2021-01-02 00:00:00Z,10,2,12,8,20,5,120
2021-01-03 00:00:00Z,10,3,13,7,30,3.3333333333333333333333333333,120
2021-01-04 00:00:00Z,10,4,14,6,40,2.5,120
2021-01-05 00:00:00Z,10,5,15,5,50,2,120


### Combining Time Series via Methods

In [ ]:
var a = TimeSeries.Factory.FromValue(1,
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 01, 05), // end
    Period.Day);

var b = TimeSeries.Factory.FromValue(2,
  new DateTime(2021, 01, 03), // start
  new DateTime(2021, 01, 07), // end
  Period.Day);

var tf = new TimeFrame();
tf["a"] = a;
tf["b"] = b;
tf["apply * 2"] = a.Apply(value => value * 2);

// join left produces a time series with the same time points as the left time series.
// note that nulls will be evaluated to null
tf["JoinLeft r + l"] = a.JoinLeft(b, (l, r) => l + r);
// Use pre defined JoinOperation to ignore nulls
tf["JoinLeft JoinOperation.Add"] = a.JoinLeft(b, JoinOperation.Add);   

// join full combines both time series
// note that nulls will be evaluated to null
tf["JoinFull r + l"] = a.JoinFull(b, (left, right) => left + right); 
// Use pre defined JoinOperation to ignore nulls
tf["JoinFull JoinOperation.Add"] = a.JoinFull(b, JoinOperation.Add);

tf

2021-01-01 00:00:00Z,1,<null>,2,<null>,1,<null>,1
2021-01-02 00:00:00Z,1,<null>,2,<null>,1,<null>,1
2021-01-03 00:00:00Z,1,2,2,3,3,3,3
2021-01-04 00:00:00Z,1,2,2,3,3,3,3
2021-01-05 00:00:00Z,1,2,2,3,3,3,3
2021-01-06 00:00:00Z,<null>,2,<null>,<null>,<null>,<null>,2
2021-01-07 00:00:00Z,<null>,2,<null>,<null>,<null>,<null>,2


### Slicing

In [ ]:
var ts = TimeSeries.Factory.FromGenerator(
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 01, 05), // end
    Period.Day,
    tp => tp.Day); // 1, 2, 3, 4, 5
 
var tf = new TimeFrame();
// sliceing by index / count
tf["ts"] = ts;
// slicing by index / count
tf["Slice(0, 2)"] = ts.Slice(0, 2);
// slicing by time range
tf["Slice(2.1, 4.1)"] = ts.Slice(new DateTime(2021, 01, 02), new DateTime(2021, 01, 04));

tf
 

2021-01-01 00:00:00Z,1,1,<null>
2021-01-02 00:00:00Z,2,2,2
2021-01-03 00:00:00Z,3,<null>,3
2021-01-04 00:00:00Z,4,<null>,4
2021-01-05 00:00:00Z,5,<null>,<null>


### Re-sampling

In [ ]:
var ts = TimeSeries.Factory.FromValue(1,
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 12, 31), // end
    Period.Day);

// down sampling
var tf = new TimeFrame();
tf["Resample Sum"] = ts.ReSample(Period.Month, AggregationType.Sum);
tf["Resample Mean"] = ts.ReSample(Period.Month, AggregationType.Mean); 
tf

2021-01-01 00:00:00Z,31,1
2021-02-01 00:00:00Z,28,1
2021-03-01 00:00:00Z,31,1
2021-04-01 00:00:00Z,30,1
2021-05-01 00:00:00Z,31,1
2021-06-01 00:00:00Z,30,1
2021-07-01 00:00:00Z,31,1
2021-08-01 00:00:00Z,31,1
2021-09-01 00:00:00Z,30,1
2021-10-01 00:00:00Z,31,1
2021-11-01 00:00:00Z,30,1


In [ ]:
// up-sampling
var tf = new TimeFrame();
tf["Resample Hour Sum"] = ts.ReSample(Period.Hour, AggregationType.Sum);
tf["Resample Hour Mean"] = ts.ReSample(Period.Hour, AggregationType.Mean);
tf

time Resample Hour Sum Resample Hour Mean 2021-01-01 00:00:00Z 0.0416666666666666666666666667 1 2021-01-01 01:00:00Z 0.0416666666666666666666666667 1 2021-01-01 02:00:00Z 0.0416666666666666666666666667 1 2021-01-01 03:00:00Z 0.0416666666666666666666666667 1 2021-01-01 04:00:00Z 0.0416666666666666666666666667 1 ... 8750 more 2021-12-31 19:00:00Z 0.0416666666666666666666666667 1 2021-12-31 20:00:00Z 0.0416666666666666666666666667 1 2021-12-31 21:00:00Z 0.0416666666666666666666666667 1 2021-12-31 22:00:00Z 0.0416666666666666666666666667 1 2021-12-31 23:00:00Z 0.0416666666666666666666666667 1

---
## .Net Interactive Integration

In [ ]:
var rnd = new Random();
var start = new DateTime(2021, 01, 01);
var end = new DateTime(2021, 12, 31, 23, 0, 0);
var tf = new TimeFrame();
var prev = (decimal?)0M;

decimal? ValueGenerator(DateTimeOffset ts)
{
    prev += (decimal)Math.Round((rnd.NextDouble() - 0.5), 4);
    return prev;
} 

for (int i = 0; i < 10; i++)
{
    prev = (decimal?)0M;
    tf.Add("ts_" + i, TimeSeries.Factory.FromGenerator(start, end, Period.Hour, ValueGenerator));
}

tf

time ts_0 ts_1 ts_2 ts_3 ts_4 ts_5 ts_6 ts_7 ts_8 ts_9 2021-01-01 00:00:00Z 0.4147 0.2535 0.4357 0.0761 -0.1864 0.3579 0.3257 -0.4551 -0.321 0.1342 2021-01-01 01:00:00Z 0.3656 0.6308 0.0330 -0.0524 -0.3749 0.3420 0.0694 -0.5172 -0.3759 0.1139 2021-01-01 02:00:00Z 0.2704 0.3231 0.2511 0.1406 -0.0529 -0.1573 0.0467 -0.9569 -0.0945 0.5021 2021-01-01 03:00:00Z 0.5940 0.6058 0.3256 0.5056 -0.2067 -0.6190 -0.4177 -0.7582 -0.1192 0.8180 2021-01-01 04:00:00Z 0.1926 0.8818 0.2360 0.4860 -0.6447 -0.8254 -0.9069 -1.0000 -0.5096 0.4767 ... 8750 more 2021-12-31 19:00:00Z 31.5408 -0.6793 -40.4796 -41.9192 3.1575 22.7842 -18.4751 -72.2393 -19.6669 52.8812 2021-12-31 20:00:00Z 31.4122 -0.2149 -40.0907 -42.2321 3.6390 22.8885 -18.2124 -72.2348 -19.7544 52.7815 2021-12-31 21:00:00Z 31.0226 0.1480 -39.9544 -42.3725 3.9439 22.8974 -18.1919 -71.8271 -19.8067 52.6589 2021-12-31 22:00:00Z 31.3038 0.0893 -40.0250 -42.5202 3.4522 22.5000 -18.3843 -72.1751 -20.0059 52.5358 2021-12-31 23:00:00Z 31.6020 0.1641 -40.4643 -42.0431 3.2907 22.0564 -18.7736 -72.4845 -19.9483 52.9346

In [ ]:
tf.Plot(chartTitle: "Random Walk", xAxisTitel: "Time (hourly)", yAxisTitle: "Value")

<!DOCTYPE html>